In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from google.cloud import bigquery

import geopy
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
from uszipcode import Zipcode
import numpy as np

In [2]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Changing the settings to override this file path in the environment
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"\Users\kenda\OneDrive\Documents\MADS\591MilestoneI\SanFranciscoBikeShare\sf-bike-321222-7e949b330c94.json"

In [4]:
# all distinct station pairs with lat lons
client = bigquery.Client()

query = """
    SELECT DISTINCT start_station_id, start_station_longitude, start_station_latitude,
        end_station_id, end_station_longitude, end_station_latitude
    FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
    WHERE EXTRACT(YEAR from start_date)=2017
"""

query_job = client.query(query)  # Make an API request.

In [5]:
latlong2017 = query_job.to_dataframe()

In [6]:
latlong2017.shape

(20027, 6)

In [7]:
## trips dataframe

query = """
    SELECT *
    FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
    WHERE EXTRACT(YEAR from start_date)=2017
"""

query_job = client.query(query)  # Make an API request.

In [8]:
trips = query_job.to_dataframe()

In [9]:
trips.shape

(519700, 21)

In [10]:
## station dataframe

client = bigquery.Client()

query = """
    SELECT *
    FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info`
"""

query_job = client.query(query)  # Make an API request.

In [11]:
stations = query_job.to_dataframe()

In [12]:
#define zipcode search function
def get_zipcode(lat, lon):
    result = search.by_coordinates(lat = lat, lng = lon, returns = 1)
    return result[0].zipcode

#add new column with generated zip-code
stations['zipcode'] = stations.apply(lambda x: get_zipcode(x.lat,x.lon), axis=1)

In [13]:
def get_tripzips(latlonDF, start_lat, start_lon, end_lat, end_lon):
    #get zipcode for each start station
    latlonDF['start_zipcode'] = latlonDF.apply(lambda x: get_zipcode(x[start_lat],x[start_lon]), axis=1)
    
    #get zipcode for each end station
    latlonDF['end_zipcode'] = latlonDF.apply(lambda x: get_zipcode(x[end_lat],x[end_lon]), axis=1)

    return latlonDF

#### Get zipcodes for unique start and end station pairs.

In [14]:
latlons = get_tripzips(latlong2017, "start_station_latitude", 'start_station_longitude', 'end_station_latitude', 'end_station_longitude')

In [15]:
latlons.head()

,start_station_id,start_station_longitude,start_station_latitude,end_station_id,end_station_longitude,end_station_latitude,start_zipcode,end_zipcode
0,232,-122.276040,37.806163,160,-122.294837,37.805318,94612,94607
1,163,-122.265320,37.797320,46,-122.242373,37.790140,94612,94606
2,178,-122.261928,37.819381,172,-122.251535,37.841800,94609,94618
3,200,-122.253810,37.800214,163,-122.265320,37.797320,94606,94612
4,277,-121.908586,37.333658,278,-121.904888,37.331932,95110,95110


In [28]:
# tag trips with start and end zipcodes
tripsWithZip = trips.merge(latlons, on = ["start_station_id", "end_station_id"])

In [29]:
#drop duplicate columns

tripsWithZip = tripsWithZip.drop(columns = ['start_station_longitude_y',
       'start_station_latitude_y', 'end_station_longitude_y',
       'end_station_latitude_y'])

##### What percentage of 2017 rides start and end in the same zip code?

In [32]:
sameZipCode = tripsWithZip[tripsWithZip["start_zipcode"]==tripsWithZip["end_zipcode"]]

In [34]:
sameZipCode.shape[0]/tripsWithZip.shape[0]

0.266823167211853

26.7% of 2017 rides start and end in the same zip code